In [ ]:
import geohash
import sqlite3
geohash.encode(51.566141,-0.009434,24)

In [ ]:
count =0


def Geocode(lat,long):
    try:
        return geohash.encode(lat,long,32)
    except:
        return ''


con = sqlite3.connect('./data/location.s3db')
con.create_function("Geocode", 2, Geocode)
cur = con.cursor()
cur.execute("update LocationReferenced set geohash = Geocode(Latitude,Longitude)")
con.commit();
con.close()



In [ ]:
count =0


def ComputeGeoString(lat,long):
    try:
        global count
        count+=1
        if count%100000==1:
            print('.')  
        return "({0},{1})".format(lat,long)
    except:
        return ''


con = sqlite3.connect('./data/location.s3db')
con.create_function("ComputeGeoString", 2, ComputeGeoString)
cur = con.cursor()
cur.execute("update LocationReferenced set Location = ComputeGeoString(Latitude,Longitude)")
con.commit();
con.close()

In [ ]:
query="select Id, Location from LocationReferenced where not GeoHash ='' order by GeoHash ;"
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()
file  = open('./data/locations.txt','w')
for row in cur.execute(query):
    file.write(row[1]+'\n')
con.close()
file.close()




In [ ]:
query="select Id, Location from LocationReferenced where not GeoHash ='' order by GeoHash ;"
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()
file  = open('./data/locations_2.txt','w')
for row in cur.execute(query):
    file.write(row[1][1:-1]+'\n')
con.close()
file.close()

In [ ]:
count =0
ligncount = 1

def Create_New_File():
    global count
    count+=1
    print(count)
    file  = open('./data/locations_sub{0}.csv'.format(count),'w')
    file.write('Id, rank, OriginId,Latitude,Longitude\n')
    return file


query="select Id, OriginId,Latitude,Longitude from LocationReferenced where not GeoHash ='' order by GeoHash ;"
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()
file  = Create_New_File()
for row in cur.execute(query):
    file.write(str(row[0])+',')
    file.write(str(ligncount)+',')
    file.write(row[1]+',')
    file.write(str(row[2])+',')
    file.write(str(row[3])+'\n')
    ligncount +=1
    if ligncount== 2000:
        ligncount = 0
        file.close()
        file = Create_New_File()
    
con.close()
file.close()
    

In [ ]:
geohash.encode(42.3357,-83.2278,32)


In [ ]:
import geolocation

In [ ]:
from geolocation import GeoLocation
from geopy.distance import great_circle
import geohash
import sqlite3

def GeoEncodeFromLocation(loc):
    return geohash.encode(loc.deg_lat, loc.deg_lon, 32)

def GetBoundingGeoCode(lat, long, dist=0.01):
    loc = GeoLocation.from_degrees(lat,long)
    SW_loc, NE_loc = loc.bounding_locations(dist)
    return sorted((GeoEncodeFromLocation(SW_loc), GeoEncodeFromLocation(NE_loc)))
    
print(geohash.encode(42.3357,-83.2278,32))    
GetBoundingGeoCode(42.3357,-83.2278, 0.02)

In [ ]:
from geolocation import GeoLocation
from geopy.distance import great_circle
import geohash
import sqlite3

initialQuery ="select Id, Latitude, Longitude from (select min(Id)as Id, Latitude, Longitude, count(CloseLocationNumber) as counted from LocationReferenced group by Latitude, Longitude order by Id) where counted==0;"
query = "select Id, Latitude, Longitude from LocationReferenced where (GeoHash>='{0}') and(GeoHash<='{1}') and not (Id={2})"
query2 = "select Id, Distance({0}, {1}, Latitude, Longitude) as distance from (select Id, Latitude, Longitude from LocationReferenced where (GeoHash>='{2}') and(GeoHash<='{3}') and not (Id={4})) where distance<={5} Order by distance;"
update = "UPDATE LocationReferenced SET CloseLocationNumber={0}, ClosestLocation={1} WHERE ID = {2};"
updateNull = "UPDATE LocationReferenced SET CloseLocationNumber=0 WHERE ID = {0};"

def GeoEncodeFromLocation(loc):
    return geohash.encode(loc.deg_lat, loc.deg_lon, 32)

def GetBoundingGeoCode(lat, long, dist=0.01):
    try:
        loc = GeoLocation.from_degrees(lat,long)
        SW_loc, NE_loc = loc.bounding_locations(dist)
        return sorted((GeoEncodeFromLocation(SW_loc), GeoEncodeFromLocation(NE_loc)))
    except Exception as Error:
        print(Error)
        return (None, None)
    

def Distance(lat1, long1, lat2, long2): 
    try:
        loc1 = (lat1, long1)
        loc2 = (lat2, long2)
        return great_circle(loc1, loc2).kilometers
    except Exception as Error:
        print(Error)
        print(lat1, long1, lat2, long2)
        return 1000

count=1 
distance = 0.01

con = sqlite3.connect('./data/location.s3db')
con.create_function("Distance", 4, Distance)
cur = con.cursor()
for row in cur.execute(initialQuery).fetchall():
    Id, Latitude, Longitude = row
    if count==1:
        print(Id, Latitude, Longitude)
    minhash, maxhash = GetBoundingGeoCode(Latitude, Longitude, distance)
    if minhash is not None:
        ClosePoints = cur.execute(query2.format( Latitude, Longitude, minhash, maxhash, Id, distance )).fetchall()
    else:
        ClosePoints =[]
    PointsNumber = len(ClosePoints)
    if not PointsNumber == 0:
        cur.execute(update.format(PointsNumber, ClosePoints[0][0],Id))
    else:
        cur.execute(updateNull.format(Id))
    con.commit()
    count+=1
    if count==2:
        print('.')
        print(Id)
        print(PointsNumber)
        if not PointsNumber == 0:
            print(ClosePoints[0][0])
    elif count == 100:
        print('..')
    elif count == 1000:
        print(Id)
        print('...')
    elif count%4000==1:
        print(Id)
        print('....')
con.close()

In [5]:
from geolocation import GeoLocation
from geopy.distance import great_circle
import geohash
import sqlite3

initialQuery ="select min(Id) as Id, min(Latitude), min(Longitude)  from LocationReferenced where CloseLocationNumber is not null and CloseLocationNumber>1 group by Geohash order by Id"
query2 = "insert into SimilarBuilding (Building1, Building2) select {4},relativeId from( select Id as relativeId, Distance({0}, {1}, Latitude, Longitude) as distance from (select Id, Latitude, Longitude from LocationReferenced where (GeoHash>='{2}') and(GeoHash<='{3}') and not (Id={4})) where distance<={5});"


def GeoEncodeFromLocation(loc):
    return geohash.encode(loc.deg_lat, loc.deg_lon, 32)

def GetBoundingGeoCode(lat, long, dist=0.01):
    try:
        loc = GeoLocation.from_degrees(lat,long)
        SW_loc, NE_loc = loc.bounding_locations(dist)
        return sorted((GeoEncodeFromLocation(SW_loc), GeoEncodeFromLocation(NE_loc)))
    except Exception as Error:
        print(Error)
        return (None, None)
    

def Distance(lat1, long1, lat2, long2): 
    try:
        loc1 = (lat1, long1)
        loc2 = (lat2, long2)
        return great_circle(loc1, loc2).kilometers
    except Exception as Error:
        print(Error)
        print(lat1, long1, lat2, long2)
        return 1000

count=1 
distance = 0.01

con = sqlite3.connect('./data/location.s3db')
con.create_function("Distance", 4, Distance)
cur = con.cursor()
for row in cur.execute(initialQuery).fetchall():
    Id, Latitude, Longitude = row
    if count==1:
        print(Id, Latitude, Longitude)
    minhash, maxhash = GetBoundingGeoCode(Latitude, Longitude, distance)
    if minhash is not None:
        try:
            ClosePoints = cur.execute(query2.format( Latitude, Longitude, minhash, maxhash, Id, distance ))
            con.commit()
        except Exception as Error:
            print(Error)
            print(query2.format( Latitude, Longitude, minhash, maxhash, Id, distance ))
    count+=1
    if count==2:
        print('.')
        print(Id)
    elif count == 100:
        print('..')
    elif count == 1000:
        print(Id)
        print('...')
    elif count%4000==1:
        print(Id)
        print('....')
con.close()


1 42.36318237000006 -83.09167672099994
.
1
..
1971
...
7476
....
14230
....
23957
....
34242
....
44380
....
59332
....
76936
....
95604
....
116884
....
139562
....
164992
....
192206
....
227452
....
264699
....
301694
....
329056
....
345991
....
365920
....
389615
....
418046
....
450093
....


In [6]:
sqlite3.sqlite_version

'3.8.11'

In [41]:
query="WITH RECURSIVE CloseBuildings(building) AS (select 1 UNION SELECT Building2 FROM SimilarBuilding, CloseBuildings where SimilarBuilding.Building1 = CloseBuildings.building), data as (SELECT min(Latitude) as Latitude_Min, min(Longitude) as Longitude_Min,max(Latitude) as Latitude_Max, max(Longitude) as Longitude_Max, min(Geohash) as RefHash FROM LocationReferenced where Id in CloseBuildings) Update LocationReferenced SET Latitude_Max= (SELECT Latitude_Max from data), Latitude_Min= (SELECT Latitude_Min from data), Longitude_Max=(SELECT Longitude_Max from data), Longitude_Min=(SELECT Longitude_Min from data), RefHash =(SELECT RefHash from data) where Id in CloseBuildings;"
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()
print(cur.execute(query).fetchall())
con.close()


[]


In [8]:
from geolocation import GeoLocation
from geopy.distance import great_circle
import geohash
import sqlite3

initialQuery ="select min(Id) as bId from LocationReferenced where LocationReferenced.Latitude_Min is null and CloseLocationNumber is not null group by Geohash"
query="WITH RECURSIVE CloseBuildings(building) AS (select {0} UNION SELECT Building2 FROM SimilarBuilding, CloseBuildings where SimilarBuilding.Building1 = CloseBuildings.building), data as (SELECT min(Latitude) as Latitude_Min, min(Longitude) as Longitude_Min,max(Latitude) as Latitude_Max, max(Longitude) as Longitude_Max, min(Geohash) as RefHash FROM LocationReferenced where Id in CloseBuildings) Update LocationReferenced SET Latitude_Max= (SELECT Latitude_Max from data), Latitude_Min= (SELECT Latitude_Min from data), Longitude_Max=(SELECT Longitude_Max from data), Longitude_Min=(SELECT Longitude_Min from data), RefHash =(SELECT RefHash from data) where Id in CloseBuildings;"

count=1 
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()

cont = True
for row in cur.execute(initialQuery).fetchall():
    Id = row[0]
    if count==1:
        print(Id)
    cur.execute(query.format(Id))
    count+=1
    con.commit()        
    if count==2:
        print('.')
        print(Id)
    elif count == 100:
        print('..')
    elif count==1000:
        print(Id)
        print('...')
    elif count%4000==1:
        print(Id)
        print('....')
con.commit()   
con.close()

453606
.
453606
..
137984
...
420418
....
4743
....
209829
....
395238
....
240428
....
269744
....
325411
....
56109
....
320905
....
210596
....
429675
....
9347
....
349905
....
335203
....
336068
....
445512
....
335143
....
354457
....
